In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd /gdrive/My\ Drive/Shell\ Hackathon/

/gdrive/My Drive/Shell Hackathon


In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import pickle
from datetime import datetime
from sklearn.svm import SVR
from   math   import radians as DegToRad       # Degrees to radians Conversion
from shapely.geometry import Point             # Imported for constraint checking
from shapely.geometry.polygon import Polygon

import warnings
warnings.filterwarnings("ignore")



In [ ]:
# class Turbine:
#     hub_height = 100.0  # unit (m)
#     rator_diameter = 100.0  # unit m
#     # surface_roughness = 0.25 * 0.001  # unit mm surface roughness
#     # surface_roughness = 0.25  # unit mm surface roughness
#     rator_radius = 0

#     # entrainment_const = 0

#     def __init__(self):
#         self.rator_radius = self.rator_diameter / 2
#         # self.entrainment_const = 0.5 / np.log(self.hub_height / self.surface_roughness)
#         return

#     # power curve
#     def P_i_X(self, v):
#         if v < 2.0:
#             return 0
#         elif v < 12.8:
#             return 0.3 * v ** 3
#         elif v < 18:
#             return 629.1
#         else:
#             return 0


In [ ]:
class LayoutGridMCGenerator:
  def __init__(self):
    return

  # rows : number of rows in wind farm
  # cols : number of columns in wind farm
  # n : number of layouts
  # N : number of turbines
  def gen_mc_grid(rows, cols, n, N, lofname):  # , xfname): generate monte carlo wind farm layout grids
    np.random.seed(seed=int(time.time()))  # init random seed
    layouts = np.zeros((n, rows * cols), dtype=np.int32)  # one row is a layout
    # layouts_cr = np.zeros((n*, 2), dtype=np.float32)  # layouts column row index
    positionX = np.random.randint(0, cols, size=(N * n * 2))
    positionY = np.random.randint(0, rows, size=(N * n * 2))
    ind_rows = 0  # index of layouts from 0 to n-1
    ind_pos = 0  # index of positionX, positionY from 0 to N*n*2-1
    # ind_crs = 0
    while ind_rows < n:
      layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
      if np.sum(layouts[ind_rows, :]) == N:
        # for ind in range(rows * cols):
        #     if layouts[ind_rows, ind] == 1:
        #         r_i = np.floor(ind / cols)
        #         c_i = np.floor(ind - r_i * cols)
        #         layouts_cr[ind_crs, 0] = c_i
        #         layouts_cr[ind_crs, 1] = r_i
        #         ind_crs += 1
        ind_rows += 1
      ind_pos += 1
      if ind_pos >= N * n * 2:
        print("Not enough positions")
        break
    # filename = "positions{}by{}by{}N{}.dat".format(rows, cols, n, N)
    np.savetxt(lofname, layouts, fmt='%d', delimiter=" ")
    # np.savetxt(xfname, layouts_cr, fmt='%d', delimiter=" ")
    # print()
    # print("generated grid layouts: "layouts.shape)
    # print(layouts)
    return layouts


  # rows : number of rows in wind farm
  # cols : number of columns in wind farm
  # n : number of layouts
  # N : number of turbines
  # generate population
  # returns layouts : (n, rows * cols) matrix with 1 for cell allocated and 0 for no turbine
  def gen_pop(rows, cols, n, N):  # generate population very similar to gen_mc_grid, just without saving layouts to a file
    np.random.seed(seed=int(time.time())) #init random seed
    layouts = np.zeros((n, rows * cols), dtype=np.int32) # one row is a layout, NA loc is 0
    positionX = np.random.randint(0, cols, size=(N * n * 2))
    positionY = np.random.randint(0, rows, size=(N * n * 2))
    ind_rows = 0  # index of layouts from 0 to n-1
    ind_pos = 0  # index of positionX, positionY from 0 to N*n*2-1
    N_count = 0

    while ind_rows < n:
      layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
      if np.sum(layouts[ind_rows, :]) == N:
        ind_rows += 1
      ind_pos += 1
      if ind_pos >= N * n * 2:
        print("Not enough positions")
        break
    return layouts


In [ ]:
class WindFarmGenetic:
  elite_rate = 0.2 # elite rate: parameter for genetic algorithm
  cross_rate = 0.6 # crossover rate: parameter for genetic algorithm
  random_rate = 0.5 # random rate: parameter for genetic algorithm
  mutate_rate = 0.1 # mutation rate: parameter for genetic algorithm
  turbine = None
  pop_size = 0 # population size : how many individuals in a population
  N = 0 # number of wind turbines
  rows = 0 # how many cell rows the wind farm are divided into
  cols = 0 # how many colus the wind farm land are divided into
  iteration = 0 # how many iterations the genetic algorithm run
  cell_width = 0  # cell width
  cell_width_half = 0  # half cell width

  def __init__(self, rows=21, cols=21, N=0, pop_size=100, iteration=200,cell_width=0, elite_rate=0.2,
                cross_rate=0.6, random_rate=0.5, mutate_rate=0.1, wind_data_file_name='wind_data_2007.csv', power_curve_file_name='power_curve.csv'):
    self.turb_specs = {   
                         'Name': 'Anon Name',
                         'Vendor': 'Anon Vendor',
                         'Type': 'Anon Type',
                         'Dia (m)': 100,
                         'Rotor Area (m2)': 7853,
                         'Hub Height (m)': 100,
                         'Cut-in Wind Speed (m/s)': 3.5,
                         'Cut-out Wind Speed (m/s)': 25,
                         'Rated Wind Speed (m/s)': 15,
                         'Rated Power (MW)': 3
                     }
    self.turb_diam = self.turb_specs['Dia (m)']
    self.turb_rad = self.turb_diam/2
    self.rows = rows
    self.cols = cols
    self.N = N
    self.pop_size = pop_size
    self.iteration = iteration

    self.cell_width = cell_width
    self.cell_width_half = cell_width * 0.5

    self.elite_rate = elite_rate
    self.cross_rate = cross_rate
    self.random_rate = random_rate
    self.mutate_rate = mutate_rate

    self.init_pop = None
    self.init_pop_nonezero_indices = None

    self.wind_data_file_name = wind_data_file_name
    self.power_curve_file_name = power_curve_file_name

    self.power_curve = self.load_power_curve(power_curve_file_name=self.power_curve_file_name)
    self.wind_inst_freq = self.bin_wind_resource_data(wind_data_file_name = self.wind_data_file_name)
    self.n_wind_instances, self.cos_dir, self.sin_dir, self.wind_sped_stacked, self.C_t = self.pre_processing(self.power_curve)

    return
  
  #generates initial population
  def gen_init_pop(self):
    self.init_pop = LayoutGridMCGenerator.gen_pop(rows=self.rows, cols=self.cols, n=self.pop_size, N=self.N)
    self.init_pop_nonezero_indices = np.zeros((self.pop_size, self.N), dtype=np.int32)
    for ind_init_pop in range(self.pop_size):
      ind_indices = 0
      for ind in range(self.rows * self.cols):
        if self.init_pop[ind_init_pop, ind] == 1:
          self.init_pop_nonezero_indices[ind_init_pop, ind_indices] = ind
          ind_indices += 1
    
    # print()
    # print("generated intial population: ")
    # print("init_pop", self.init_pop)
    # print("init_pop_nonezero_indices", self.init_pop_nonezero_indices)
    return

  def save_init_pop(self, fname):
    np.savetxt(fname, self.init_pop, fmt="%d", delimiter=" ")
    return

  def search_sorted(self, lookup, sample_array):
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Returns lookup indices for closest values w.r.t sample_array elements
    
    :called_from
        preProcessing, getAEP
    
    :param
        lookup       - The lookup array
        sample_array - Array, whose elements need to be matched
                       against lookup elements. 
        
    :return
        lookup indices for closest values w.r.t sample_array elements 
    """
    lookup_middles = lookup[1:] - np.diff(lookup.astype('f'))/2
    idx1 = np.searchsorted(lookup_middles, sample_array)
    indices = np.arange(lookup.shape[0])[idx1]
    return indices

  def load_power_curve(self, power_curve_file_name):
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Returns a 2D numpy array with information about
    turbine thrust coeffecient and power curve of the 
    turbine for given wind speed
    
    :called_from
        main function
    
    :param
        power_curve_file_name - power curve csv file location
        
    :return
        Returns a 2D numpy array with cols Wind Speed (m/s), 
        Thrust Coeffecient (non dimensional), Power (MW)
    """
    powerCurve = pd.read_csv(power_curve_file_name, sep=',', dtype = np.float32)
    powerCurve = powerCurve.to_numpy(dtype = np.float32)
    return(powerCurve)

  def bin_wind_resource_data(self, wind_data_file_name):
    r"""
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Loads the wind data. Returns a 2D array with shape (36,15). 
    Each cell in  array is a wind direction and speed 'instance'. 
    Values in a cell correspond to probability of instance
    occurence.  
    
    :Called from
        main function
        
    :param
        wind_data_file_name - Wind Resource csv file  
        
    :return
        1-D flattened array of the 2-D array shown below. Values 
        inside cells, rough probabilities of wind instance occurence. 
        Along: Row-direction (drct), Column-Speed (s). Array flattened
        for vectorization purpose. 
        
                      |0<=s<2|2<=s<4| ...  |26<=s<28|28<=s<30|
        |_____________|______|______|______|________|________|
        | drct = 360  |  --  |  --  |  --  |   --   |   --   |
        | drct = 10   |  --  |  --  |  --  |   --   |   --   |
        | drct = 20   |  --  |  --  |  --  |   --   |   --   |
        |   ....      |  --  |  --  |  --  |   --   |   --   |
        | drct = 340  |  --  |  --  |  --  |   --   |   --   |
        | drct = 350  |  --  |  --  |  --  |   --   |   --   |        
    """
    
    # Load wind data. Then, extracts the 'drct', 'sped' columns
    df = pd.read_csv(wind_data_file_name)
    wind_resource = df[['drct', 'sped']].to_numpy(dtype = np.float32)
    
    # direction 'slices' in degrees
    slices_drct   = np.roll(np.arange(10, 361, 10, dtype=np.float32), 1)
    ## slices_drct   = [360, 10.0, 20.0.......340, 350]
    n_slices_drct = slices_drct.shape[0]
    
    # speed 'slices'
    slices_sped   = [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 
                        18.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0]
    n_slices_sped = len(slices_sped)-1

    
    # placeholder for binned wind
    binned_wind = np.zeros((n_slices_drct, n_slices_sped), 
                           dtype = np.float32)
    
    # 'trap' data points inside the bins. 
    for i in range(n_slices_drct):
      for j in range(n_slices_sped):      
        # because we already have drct in the multiples of 10
        foo = wind_resource[(wind_resource[:,0] == slices_drct[i])] 

        foo = foo[(foo[:,1] >= slices_sped[j]) 
                      & (foo[:,1] <  slices_sped[j+1])]
        
        binned_wind[i,j] = foo.shape[0]  
  
    wind_inst_freq   = binned_wind/np.sum(binned_wind)
    wind_inst_freq   = wind_inst_freq.ravel()
    
    return(wind_inst_freq)

  def pre_processing(self, power_curve):
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Doing preprocessing to avoid the same repeating calculations.
    Record the required data for calculations. Do that once.
    Data are set up (shaped) to assist vectorization. Used later in
    function totalAEP. 
    
    :called_from
        main function
    
    :param
        power_curve - 2D numpy array with cols Wind Speed (m/s), 
                      Thrust Coeffecient (non dimensional), Power (MW)
        
    :return
        n_wind_instances  - number of wind instances (int)
        cos_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        sin_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        wind_sped_stacked - column staked all speed instances n_turb times. 
        C_t               - 3D array with shape (n_wind_instances, n_turbs, n_turbs)
                            Value changing only along axis=0. C_t, thrust coeff.
                            values for all speed instances. 
    """
    # number of turbines
    n_turbs       =   50
    
    # direction 'slices' in degrees
    slices_drct   = np.roll(np.arange(10, 361, 10, dtype=np.float32), 1)
    ## slices_drct   = [360, 10.0, 20.0.......340, 350]
    n_slices_drct = slices_drct.shape[0]
    
    # speed 'slices'
    slices_sped   = [0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 
                        18.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0]
    n_slices_sped = len(slices_sped)-1
    
    # number of wind instances
    n_wind_instances = (n_slices_drct)*(n_slices_sped)
    
    # Create wind instances. There are two columns in the wind instance array
    # First Column - Wind Speed. Second Column - Wind Direction
    # Shape of wind_instances (n_wind_instances,2). 
    # Values [1.,360.],[3.,360.],[5.,360.]...[25.,350.],[27.,350.],29.,350.]
    wind_instances = np.zeros((n_wind_instances,2), dtype=np.float32)
    counter = 0
    for i in range(n_slices_drct):
      for j in range(n_slices_sped):
        wind_drct =  slices_drct[i]
        wind_sped = (slices_sped[j] + slices_sped[j+1])/2
        
        wind_instances[counter,0] = wind_sped
        wind_instances[counter,1] = wind_drct
        counter += 1

	    # So that the wind flow direction aligns with the +ve x-axis.			
    # Convert inflow wind direction from degrees to radians
    wind_drcts =  np.radians(wind_instances[:,1] - 90)
    # For coordinate transformation 
    cos_dir = np.cos(wind_drcts).reshape(n_wind_instances,1)
    sin_dir = np.sin(wind_drcts).reshape(n_wind_instances,1)
    
    # create copies of n_wind_instances wind speeds from wind_instances
    wind_sped_stacked = np.column_stack([wind_instances[:,0]]*n_turbs)
   
    # Pre-prepare matrix with stored thrust coeffecient C_t values for 
    # n_wind_instances shape (n_wind_instances, n_turbs, n_turbs). 
    # Value changing only along axis=0. C_t, thrust coeff. values for all 
    # speed instances.
    # we use power_curve data as look up to estimate the thrust coeff.
    # of the turbine for the corresponding closest matching wind speed
    indices = self.search_sorted(power_curve[:,0], wind_instances[:,0])
    C_t     = power_curve[indices,1]
    # stacking and reshaping to assist vectorization
    C_t     = np.column_stack([C_t]*(n_turbs*n_turbs))
    C_t     = C_t.reshape(n_wind_instances, n_turbs, n_turbs)
    
    return(n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t)


  def get_layout_power(self, turb_rad, turb_coords, power_curve, wind_inst_freq, 
            n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t):
    
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Calculates power for each turbine for a given layout of the wind farm. Vectorised version.
    
    :called from
        main
        
    :param
        turb_rad         - Radius of the turbine (m)
        turb_coords       - 2D array turbine euclidean x,y coordinates
        power_curve       - For estimating power. 
        wind_inst_freq    - 1-D flattened with rough probabilities of 
                            wind instance occurence.
                            n_wind_instances  - number of wind instances (int)
        cos_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        sin_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        wind_sped_stacked - column staked all speed instances n_turb times. 
        C_t               - 3D array with shape (n_wind_instances, n_turbs, n_turbs)
                            Value changing only along axis=0. C_t, thrust coeff.
                            values for all speed instances. 
    
    :return
        wind farm AEP in Gigawatt Hours, GWh (float)
    """
    # number of turbines
    n_turbs        =   turb_coords.shape[0]
    assert n_turbs ==  50, "Error! Number of turbines is not 50."
    
    # Prepare the rotated coordinates wrt the wind direction i.e downwind(x) & crosswind(y) 
    # coordinates wrt to the wind direction for each direction in wind_instances array
    rotate_coords   =  np.zeros((n_wind_instances, n_turbs, 2), dtype=np.float32)
    # Coordinate Transformation. Rotate coordinates to downwind, crosswind coordinates
    rotate_coords[:,:,0] =  np.matmul(cos_dir, np.transpose(turb_coords[:,0].reshape(n_turbs,1))) - \
                           np.matmul(sin_dir, np.transpose(turb_coords[:,1].reshape(n_turbs,1)))
    rotate_coords[:,:,1] =  np.matmul(sin_dir, np.transpose(turb_coords[:,0].reshape(n_turbs,1))) +\
                           np.matmul(cos_dir, np.transpose(turb_coords[:,1].reshape(n_turbs,1)))
 
    
    # x_dist - x dist between turbine pairs wrt downwind/crosswind coordinates)
    # for each wind instance
    x_dist = np.zeros((n_wind_instances,n_turbs,n_turbs), dtype=np.float32)
    for i in range(n_wind_instances):
      tmp = rotate_coords[i,:,0].repeat(n_turbs).reshape(n_turbs, n_turbs)
      x_dist[i] = tmp - tmp.transpose()
    

    # y_dist - y dist between turbine pairs wrt downwind/crosswind coordinates)
    # for each wind instance    
    y_dist = np.zeros((n_wind_instances,n_turbs,n_turbs), dtype=np.float32)
    for i in range(n_wind_instances):
      tmp = rotate_coords[i,:,1].repeat(n_turbs).reshape(n_turbs, n_turbs)
      y_dist[i] = tmp - tmp.transpose()
    y_dist = np.abs(y_dist) 
     

    # Now use element wise operations to calculate speed deficit.
    # kw, wake decay constant presetted to 0.05
    # use the jensen's model formula. 
    # no wake effect of turbine on itself. either j not an upstream or wake 
    # not happening on i because its outside of the wake region of j
    # For some values of x_dist here RuntimeWarning: divide by zero may occur
    # That occurs for negative x_dist. Those we anyway mark as zeros. 
    sped_deficit = (1-np.sqrt(1-C_t))*((turb_rad/(turb_rad + 0.05*x_dist))**2) 
    sped_deficit[((x_dist <= 0) | ((x_dist > 0) & (y_dist > (turb_rad + 0.05*x_dist))))] = 0.0
    
    # Calculate Total speed deficit from all upstream turbs, using sqrt of sum of sqrs
    sped_deficit_eff  = np.sqrt(np.sum(np.square(sped_deficit), axis = 2))

    # Element wise multiply the above with (1- sped_deficit_eff) to get
    # effective windspeed due to the happening wake
    wind_sped_eff     = wind_sped_stacked*(1.0-sped_deficit_eff)

    # Estimate power from power_curve look up for wind_sped_eff
    indices = self.search_sorted(power_curve[:,0], wind_sped_eff.ravel())
    power   = power_curve[indices,2]
    power   = power.reshape(n_wind_instances,n_turbs)
    
    q = np.expand_dims(wind_inst_freq, axis=1)
    
    # multiply the respective values with the wind instance probabilities 
    ## year_hours = 8760.0
    turbines_power = power*q
    turbines_power = np.sum(turbines_power, axis=0)
    
    # Convert MWh to GWh
    # turbines_power = turbines_power/1e3
    
    return(turbines_power)

  #calculate fitness value of population
  def mc_fitness(self, pop, rows, cols, pop_size, N, lp):
    for i in range(pop_size):
      xy_position = np.zeros((2, N), dtype=np.float32) # x y position
      cr_position = np.zeros((2, N), dtype=np.int32)  # column row position
      ind_position = np.zeros(N, dtype=np.int32)
      ind_pos = 0
      for ind in range(rows * cols):
        if pop[i, ind] == 1:
          r_i = np.floor(ind / cols)
          c_i = np.floor(ind - r_i * cols)
          cr_position[0, ind_pos] = c_i
          cr_position[1, ind_pos] = r_i
          xy_position[0, ind_pos] = c_i * self.cell_width + self.cell_width_half
          xy_position[1, ind_pos] = r_i * self.cell_width + self.cell_width_half
          ind_position[ind_pos] = ind
          ind_pos += 1

      # print(init_pops_data_folder)
      # print("pop_index", i)
      # print("xy_position: ", xy_position.shape)
      # print(xy_position)
      # print("cr_position: ", cr_position.shape)
      # print(cr_position)

      # lp_power_accum = np.zeros(N, dtype=np.float32)  # a specific layout power accumulate
      power_curve = self.load_power_curve(power_curve_file_name=self.power_curve_file_name) # Load the power curve

      # Pass wind data csv file location to function binWindResourceData.
      # Retrieve probabilities of wind instance occurence.
      wind_inst_freq = self.bin_wind_resource_data(wind_data_file_name=self.wind_data_file_name)

      # Doing preprocessing to avoid the same repeating calculations. Record 
      # the required data for calculations. Do that once. Data are set up (shaped)
      # to assist vectorization. Used later in function totalAEP.
      n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = self.pre_processing(power_curve)

      # # check if there is any constraint is violated before we do anything. Comment 
      # # out the function call to checkConstraints below if you desire. Note that 
      # # this is just a check and the function does not quantifies the amount by 
      # # which the constraints are violated if any. 
      # checkConstraints(turb_coords, turb_diam)
      lp_power_accum = self.get_layout_power(self.turb_rad, xy_position.T, power_curve, wind_inst_freq, 
                  n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t)
      lp[i, ind_position] = lp_power_accum

    # print()
    # print("layout_power", lp.shape)
    # print(lp)
    return



  # generate the location index coordinate and average power output at each location index coordinate
  # location index coordinate : in the cells, the cell with index 1 has location index (0,0) and the cell 2 has (1,0)
  # store the location index coordinate in x.dat and average power in y.dat
  def mc_gen_xy(self, rows, cols, layouts, n, N, xfname, yfname):
    layouts_cr = np.zeros((rows * cols, 2), dtype=np.int32) # layouts column row index
    n_copies = np.sum(layouts, axis=0)
    layouts_power = np.zeros((n, rows * cols), dtype=np.float32)
    self.mc_fitness(pop=layouts, rows=rows, cols=cols, pop_size=n, N=N, lp=layouts_power)
    sum_layout_power = np.sum(layouts_power, axis=0)
    mean_power = np.zeros(rows * cols, dtype=np.float32)
    for i in range(rows * cols):
      if n_copies[i]>0:
        mean_power[i] = sum_layout_power[i] / n_copies[i]
    for ind in range(rows * cols):
      r_i = np.floor(ind / cols)
      c_i = np.floor(ind - r_i * cols)
      layouts_cr[ind, 0] = c_i
      layouts_cr[ind, 1] = r_i
    np.savetxt(xfname, layouts_cr, fmt='%d', delimiter=" ")
    np.savetxt(yfname, mean_power, fmt='%f', delimiter=" ")

    # print()
    # print("layouts_cr_postion", layouts_cr.shape)
    # print(layouts_cr)
    # print("mean_power", mean_power.shape)
    # print(mean_power)
    return


  def load_init_pop(self, fname):
    self.init_pop = np.genfromtxt(fname, delimiter=" ", dtype=np.int32)
    self.init_pop_nonezero_indices = np.zeros((self.pop_size, self.N), dtype=np.int32)
    for ind_init_pop in range(self.pop_size):
      ind_indices = 0
      for ind in range(self.rows * self.cols):
        if self.init_pop[ind_init_pop, ind] == 1:
          self.init_pop_nonezero_indices[ind_init_pop, ind_indices] = ind
          ind_indices += 1
    return


  #calculate fitness value
  def sugga_fitness(self, pop, rows, cols, pop_size, N, po):
    fitness_val = np.zeros(pop_size, dtype=np.float32)
    for i in range(pop_size):
      # layout = np.reshape(pop[i, :], newshape=(rows, cols))
      xy_position = np.zeros((2, N), dtype=np.float32)  # x y position
      cr_position = np.zeros((2, N), dtype=np.int32)  # column row position
      ind_position = np.zeros(N, dtype=np.int32)
      ind_pos = 0
      for ind in range(rows * cols):
        if pop[i, ind] == 1:
          r_i = np.floor(ind / cols)
          c_i = np.floor(ind - r_i * cols)
          cr_position[0, ind_pos] = c_i
          cr_position[1, ind_pos] = r_i
          xy_position[0, ind_pos] = c_i * self.cell_width + self.cell_width_half
          xy_position[1, ind_pos] = r_i * self.cell_width + self.cell_width_half
          ind_position[ind_pos] = ind
          ind_pos += 1
      # lp_power_accum = np.zeros(N, dtype=np.float32)  # a specific layout power accumulate
      # power_curve = self.load_power_curve(power_curve_file_name=self.power_curve_file_name) # Load the power curve

      # Pass wind data csv file location to function binWindResourceData.
      # Retrieve probabilities of wind instance occurence.
      # wind_inst_freq = self.bin_wind_resource_data(wind_data_file_name=self.wind_data_file_name)

      # Doing preprocessing to avoid the same repeating calculations. Record 
      # the required data for calculations. Do that once. Data are set up (shaped)
      # to assist vectorization. Used later in function totalAEP.
      # n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = self.pre_processing(power_curve)

      # # check if there is any constraint is violated before we do anything. Comment 
      # # out the function call to checkConstraints below if you desire. Note that 
      # # this is just a check and the function does not quantifies the amount by 
      # # which the constraints are violated if any. 
      # checkConstraints(turb_coords, turb_diam)
      lp_power_accum = self.get_layout_power(self.turb_rad, xy_position.T, self.power_curve, self.wind_inst_freq, 
                  self.n_wind_instances, self.cos_dir, self.sin_dir, self.wind_sped_stacked, self.C_t)
      sorted_index = np.argsort(lp_power_accum)  # power from least to largest
      po[i, :] = ind_position[sorted_index]
      fitness_val[i] = np.sum(lp_power_accum)
      #
    return fitness_val

  def sugga_move_worst(self, rows, cols, pop, pop_indices, pop_size, power_order, mars=None,svr_model=None):
    np.random.seed(seed=int(time.time()))
    for i in range(pop_size):
      r = np.random.randn()
      if r < 0.5:
        self.sugga_move_worst_case_random(i=i, rows=rows, cols=cols, pop=pop, pop_indices=pop_indices,
                                          pop_size=pop_size, power_order=power_order)
      else:
        self.sugga_move_worst_case_best(i=i, rows=rows, cols=cols, pop=pop, pop_indices=pop_indices,
                                        pop_size=pop_size, power_order=power_order, mars=mars, svr_model=svr_model)
    return

  def sugga_move_worst_case_random(self, i, rows, cols, pop, pop_indices, pop_size, power_order):
    np.random.seed(seed=int(time.time()))
    turbine_pos = power_order[i, 0]
    while True:
      null_turbine_pos = np.random.randint(0, cols * rows)
      if(pop[i, null_turbine_pos]==0):
        break
    pop[i, turbine_pos] = 0
    pop[i, null_turbine_pos] = 1
    power_order[i, 0] = null_turbine_pos
    pop_indices[i, :] = np.sort(power_order[i, :])
    return

  def sugga_move_worst_case_best(self, i, rows, cols, pop, pop_indices, pop_size, power_order, mars,svr_model):
    np.random.seed(seed=int(time.time()))
    n_candiate = 5
    pos_candidate = np.zeros((n_candiate, 2), dtype=np.int32)
    ind_pos_candidate = np.zeros(n_candiate, dtype=np.int32)
    turbine_pos = power_order[i, 0]
    ind_can = 0
    while True:
      null_turbine_pos = np.random.randint(0, cols * rows)
      if(pop[i, null_turbine_pos]==0):
        pos_candidate[ind_can, 1] = int(np.floor(null_turbine_pos / cols))
        pos_candidate[ind_can, 0] = int(np.floor(null_turbine_pos - pos_candidate[ind_can, 1] * cols))
        ind_pos_candidate[ind_can] = null_turbine_pos
        ind_can += 1
        if ind_can == n_candiate:
          break
    svr_val = svr_model.predict(pos_candidate)
    sorted_index = np.argsort(-svr_val)  # fitness value descending from largest to least
    innd = 0
    while True:
      null_turbine_pos = ind_pos_candidate[sorted_index[innd]]
      if(pop[i, null_turbine_pos]==0):
        break
      innd += 1

    pop[i, turbine_pos] = 0
    pop[i, null_turbine_pos] = 1

    power_order[i, 0] = null_turbine_pos
    pop_indices[i, :] = np.sort(power_order[i, :])
    return

  def sugga_select(self, pop, pop_indices, pop_size, elite_rate, random_rate):
    n_elite = int(pop_size * elite_rate)
    parents_ind = [i for i in range(n_elite)]
    np.random.seed(seed=int(time.time()))  # init random seed
    for i in range(n_elite, pop_size):
      if np.random.randn() < random_rate:
        parents_ind.append(i)
    parent_layouts = pop[parents_ind, :]
    parent_pop_indices = pop_indices[parents_ind, :]
    return len(parent_pop_indices), parent_layouts, parent_pop_indices
  
  def sugga_crossover(self, N, pop, pop_indices, pop_size, n_parents, parent_layouts, parent_pop_indices):
    n_counter = 0
    np.random.seed(seed=int(time.time()))  # init random seed
    while n_counter < pop_size:
      male = np.random.randint(0, n_parents)
      female = np.random.randint(0, n_parents)
      if male != female:
        cross_point = np.random.randint(1, N)
        if parent_pop_indices[male, cross_point - 1] < parent_pop_indices[female, cross_point]:
          pop[n_counter, :] = 0
          pop[n_counter, :parent_pop_indices[male, cross_point - 1] + 1] = parent_layouts[male, :parent_pop_indices[male, cross_point - 1] + 1]
          pop[n_counter, parent_pop_indices[female, cross_point]:] = parent_layouts[female, parent_pop_indices[female, cross_point]:]

          pop_indices[n_counter, :cross_point] = parent_pop_indices[male, :cross_point]
          pop_indices[n_counter, cross_point:] = parent_pop_indices[female, cross_point:]
          n_counter += 1
    return

  # SUGGA mutation
  def sugga_mutation(self, rows, cols, N, pop, pop_indices, pop_size, mutation_rate):
    np.random.seed(seed=int(time.time()))
    for i in range(pop_size):
      if np.random.randn() > mutation_rate:
        continue
      while True:
        turbine_pos = np.random.randint(0, cols * rows)
        if pop[i, turbine_pos] == 1:
          break
      while True:
        null_turbine_pos = np.random.randint(0, cols * rows)
        if pop[i, null_turbine_pos] == 0:
          break
      pop[i, turbine_pos] = 0
      pop[i, null_turbine_pos] = 1

      for j in range(N):
        if pop_indices[i, j] == turbine_pos:
          pop_indices[i, j] = null_turbine_pos
          break
      pop_indices[i, :] = np.sort(pop_indices[i, :])
    return



  # SUGGA: support vector regression guided genetic algorithm
  def sugga_genetic_algo(self, ind_time=0, svr_model=None, result_folder=None):
    # start_time = self.cal_P_rate_totoal()
    start_time = datetime.now()
    print("Support vector regression guided genetic algorithm starts....")
    fitness_generations = np.zeros(self.iteration, dtype=np.float32)  # best fitness value in each generation
    best_layout_generations = np.zeros((self.iteration, self.rows * self.cols),
                                        dtype=np.int32)  # best layout in each generation
    power_order = np.zeros((self.pop_size, self.N),
                            dtype=np.int32)  # each row is a layout cell indices. in each layout, order turbine power from least to largest
    pop = np.copy(self.init_pop)
    pop_indices = np.copy(self.init_pop_nonezero_indices)  # each row is a layout cell indices.
    eN = int(np.floor(self.pop_size * self.elite_rate))  # elite number
    rN = int(int(np.floor(self.pop_size * self.mutate_rate)) / eN) * eN  # reproduce number
    mN = rN  # mutation number
    cN = self.pop_size - eN - mN  # crossover number

    for gen in range(self.iteration):
      print("generation {}...".format(gen))
      fitness_value = self.sugga_fitness(pop=pop, rows=self.rows, cols=self.cols, pop_size=self.pop_size,
                                         N=self.N, po=power_order)
      sorted_index = np.argsort(-fitness_value)  # fitness value descending from largest to least
      pop = pop[sorted_index, :]
      power_order = power_order[sorted_index, :]
      pop_indices = pop_indices[sorted_index, :]
      if gen == 0:
        fitness_generations[gen] = fitness_value[sorted_index[0]]
        best_layout_generations[gen, :] = pop[0, :]
      else:
        if fitness_value[sorted_index[0]] > fitness_generations[gen - 1]:
          fitness_generations[gen] = fitness_value[sorted_index[0]]
          best_layout_generations[gen, :] = pop[0, :]
        else:
          fitness_generations[gen] = fitness_generations[gen - 1]
          best_layout_generations[gen, :] = best_layout_generations[gen - 1, :]
      
      # print()
      print("best layout for generation ", best_layout_generations[gen])
      print(self.AEP(layout=best_layout_generations[gen], N=self.N))

      self.sugga_move_worst(rows=self.rows, cols=self.cols, pop=pop, pop_indices=pop_indices,
                            pop_size=self.pop_size, power_order=power_order, svr_model=svr_model)
      n_parents, parent_layouts, parent_pop_indices = self.sugga_select(pop=pop, pop_indices=pop_indices, pop_size=self.pop_size,
                                                                        elite_rate=self.elite_rate, random_rate=self.random_rate)
      self.sugga_crossover(N=self.N, pop=pop, pop_indices=pop_indices, pop_size=self.pop_size, n_parents=n_parents,
                           parent_layouts=parent_layouts, parent_pop_indices=parent_pop_indices)
      self.sugga_mutation(rows=self.rows, cols=self.cols, N=self.N, pop=pop, pop_indices=pop_indices,
                          pop_size=self.pop_size, mutation_rate=self.mutate_rate)
    print("best_layout: ", self.AEP(layout=best_layout_generations[gen], N=self.N))  
    end_time = datetime.now()
    run_time = (end_time - start_time).total_seconds()
    # eta_generations = np.copy(fitness_generations)
    # eta_generations = eta_generations * (1.0 / P_rate_total)
    time_stamp = datetime.now().strftime("%Y%m%d%H%M%S")

    # filename = "{}/sugga_eta_N{}_{}_{}.dat".format(result_folder,self.N, ind_time, time_stamp)
    # np.savetxt(filename, eta_generations, fmt='%f', delimiter="  ")
    filename = "{}/sugga_best_layouts_N{}_{}.dat".format(result_folder,self.N, ind_time)
    np.savetxt(filename, best_layout_generations, fmt='%d', delimiter=" ")
    # filename = "{}/sugga_best_layouts_NA_N{}_{}_{}.dat".format(result_folder,self.N, ind_time, time_stamp)
    # np.savetxt(filename, best_layout_NA_generations, fmt='%d', delimiter="  ")
    print("Support vector regression guided genetic algorithm ends.")
    filename = "{}/sugga_runtime.txt".format(result_folder)
    f = open(filename, "a+")
    f.write("{}\n".format(run_time))
    f.close()
    # filename = "{}/sugga_eta.txt".format(result_folder)
    # f = open(filename, "a+")
    # f.write("{}\n".format(eta_generations[self.iteration - 1]))
    # f.close()
    return run_time#, eta_generations[self.iteration - 1]
  
  def get_AEP(self, turb_rad, turb_coords, power_curve, wind_inst_freq,
             n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t):
    
    """
    -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-
    
    Calculates AEP of the wind farm. Vectorised version.
    
    :called from
        main
        
    :param
        turb_diam         - Radius of the turbine (m)
        turb_coords       - 2D array turbine euclidean x,y coordinates
        power_curve       - For estimating power. 
        wind_inst_freq    - 1-D flattened with rough probabilities of 
                            wind instance occurence.
                            n_wind_instances  - number of wind instances (int)
        cos_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        sin_dir           - For coordinate transformation 
                            2D Array. Shape (n_wind_instances,1)
        wind_sped_stacked - column staked all speed instances n_turb times. 
        C_t               - 3D array with shape (n_wind_instances, n_turbs, n_turbs)
                            Value changing only along axis=0. C_t, thrust coeff.
                            values for all speed instances. 
    
    :return
        wind farm AEP in Gigawatt Hours, GWh (float)
    """
    # number of turbines
    n_turbs        =   turb_coords.shape[0]
    assert n_turbs ==  50, "Error! Number of turbines is not 50."
    
    # Prepare the rotated coordinates wrt the wind direction i.e downwind(x) & crosswind(y) 
    # coordinates wrt to the wind direction for each direction in wind_instances array
    rotate_coords   =  np.zeros((n_wind_instances, n_turbs, 2), dtype=np.float32)
    # Coordinate Transformation. Rotate coordinates to downwind, crosswind coordinates
    rotate_coords[:,:,0] =  np.matmul(cos_dir, np.transpose(turb_coords[:,0].reshape(n_turbs,1))) - \
                           np.matmul(sin_dir, np.transpose(turb_coords[:,1].reshape(n_turbs,1)))
    rotate_coords[:,:,1] =  np.matmul(sin_dir, np.transpose(turb_coords[:,0].reshape(n_turbs,1))) +\
                           np.matmul(cos_dir, np.transpose(turb_coords[:,1].reshape(n_turbs,1)))
 
    
    # x_dist - x dist between turbine pairs wrt downwind/crosswind coordinates)
    # for each wind instance
    x_dist = np.zeros((n_wind_instances,n_turbs,n_turbs), dtype=np.float32)
    for i in range(n_wind_instances):
      tmp = rotate_coords[i,:,0].repeat(n_turbs).reshape(n_turbs, n_turbs)
      x_dist[i] = tmp - tmp.transpose()
    

    # y_dist - y dist between turbine pairs wrt downwind/crosswind coordinates)
    # for each wind instance    
    y_dist = np.zeros((n_wind_instances,n_turbs,n_turbs), dtype=np.float32)
    for i in range(n_wind_instances):
      tmp = rotate_coords[i,:,1].repeat(n_turbs).reshape(n_turbs, n_turbs)
      y_dist[i] = tmp - tmp.transpose()
    y_dist = np.abs(y_dist) 
     

    # Now use element wise operations to calculate speed deficit.
    # kw, wake decay constant presetted to 0.05
    # use the jensen's model formula. 
    # no wake effect of turbine on itself. either j not an upstream or wake 
    # not happening on i because its outside of the wake region of j
    # For some values of x_dist here RuntimeWarning: divide by zero may occur
    # That occurs for negative x_dist. Those we anyway mark as zeros. 
    sped_deficit = (1-np.sqrt(1-C_t))*((turb_rad/(turb_rad + 0.05*x_dist))**2) 
    sped_deficit[((x_dist <= 0) | ((x_dist > 0) & (y_dist > (turb_rad + 0.05*x_dist))))] = 0.0
    
    
    # Calculate Total speed deficit from all upstream turbs, using sqrt of sum of sqrs
    sped_deficit_eff  = np.sqrt(np.sum(np.square(sped_deficit), axis = 2))

    
    # Element wise multiply the above with (1- sped_deficit_eff) to get
    # effective windspeed due to the happening wake
    wind_sped_eff     = wind_sped_stacked*(1.0-sped_deficit_eff)

    
    # Estimate power from power_curve look up for wind_sped_eff
    indices = self.search_sorted(power_curve[:,0], wind_sped_eff.ravel())
    power   = power_curve[indices,2]
    power   = power.reshape(n_wind_instances,n_turbs)
    
    # Farm power for single wind instance 
    power   = np.sum(power, axis=1)
    
    # multiply the respective values with the wind instance probabilities 
    # year_hours = 8760.0
    AEP = 8760.0*np.sum(power*wind_inst_freq)
    
    # Convert MWh to GWh
    AEP = AEP/1e3
    
    return(AEP)

  def AEP(self, layout, N):
    xy_position = np.zeros((2, N), dtype=np.float32)
    ind_pos = 0
    for ind in range(self.rows * self.cols):
      if layout[ind] == 1:
        r_i = np.floor(ind / self.cols)
        c_i = np.floor(ind - r_i * self.cols)
        xy_position[0, ind_pos] = c_i * self.cell_width + self.cell_width_half
        xy_position[1, ind_pos] = r_i * self.cell_width + self.cell_width_half
        # ind_position[ind_pos] = ind
        ind_pos += 1
    # lp_power_accum = np.zeros(N, dtype=np.float32)  # a specific layout power accumulate
    power_curve = self.load_power_curve(power_curve_file_name=self.power_curve_file_name) # Load the power curve

    # Pass wind data csv file location to function binWindResourceData.
    # Retrieve probabilities of wind instance occurence.
    wind_inst_freq = self.bin_wind_resource_data(wind_data_file_name=self.wind_data_file_name)

    # Doing preprocessing to avoid the same repeating calculations. Record 
    # the required data for calculations. Do that once. Data are set up (shaped)
    # to assist vectorization. Used later in function totalAEP.
    n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = self.pre_processing(power_curve)

    # # check if there is any constraint is violated before we do anything. Comment 
    # # out the function call to checkConstraints below if you desire. Note that 
    # # this is just a check and the function does not quantifies the amount by 
    # # which the constraints are violated if any. 
    # checkConstraints(turb_coords, turb_diam)

    return self.get_AEP(self.turb_rad, xy_position.T, power_curve, wind_inst_freq, 
                  n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t)
    
    

In [ ]:
elite_rate = 0.2 # elite rate: parameter for genetic algorithm
cross_rate = 0.6 # crossover rate: parameter for genetic algorithm
random_rate = 0.5 # random rate: parameter for genetic algorithm
mutate_rate = 0.1 # mutation rate: parameter for genetic algorithm

wt_N =  50 # number of wind turbines

population_size = 120  # how many layouts in a population
iteration_times = 300  # how many iterations in a genetic algorithm run

n_inits = 10  # number of initial populations n_inits >= run_times
run_times = 2 # number of different initial populations

# wind farm size, cells
cols_cells = 10  # number of cells each row
rows_cells = 10  # number of cells each column
cell_width = 400 # unit : m

wind_data_file_name = "merged_wind_data.csv"

# all data will be save in data folder
data_folder = "data"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

wfg = WindFarmGenetic(rows=rows_cells, cols=cols_cells, N=wt_N, pop_size=population_size,
                                      iteration=iteration_times, cell_width=cell_width, elite_rate=elite_rate,
                                             cross_rate=cross_rate, random_rate=random_rate, mutate_rate=mutate_rate, wind_data_file_name=wind_data_file_name)

# **Generate Intial Populations**

In [ ]:
# initial population saved folder
init_pops_data_folder = "{}/init_data".format(data_folder)
# print(init_pops_data_folder)
if not os.path.exists(init_pops_data_folder):
  os.makedirs(init_pops_data_folder)

# generate initial populations to start with and store them
# in order to start from the same initial population for different methods
# so it is fair to compare the final results
# for i in range(n_inits):
#   wfg.gen_init_pop()
#   wfg.save_init_pop("{}/init_{}.dat".format(init_pops_data_folder,i))


# **Creates results folder**

In [ ]:
# results folder
results_data_folder = "data/results"
if not os.path.exists(results_data_folder):
  os.makedirs(results_data_folder)
# if sg folder does not exist, create these folders. Folders to store the running results
# sg: support vector regression guided genetic algorithm


sg_result_folder = "{}/sg".format(results_data_folder)
if not os.path.exists(sg_result_folder):
  os.makedirs(sg_result_folder)
    
# resul_arr: run_times by 2 , the first column is the run time in seconds for each run and the second column is the conversion efficiency for the run
result_arr = np.zeros((run_times, 2), dtype=np.float32)

 # **Run SUpport vector regression Guided Genetic Algorithm (SUGGA)**

 **Generate wind distribution surface**


In [ ]:
n_mc_samples = 2000  # svr train data, number of layouts to average

wds_data_folder = "{}/wds".format(data_folder)
if not os.path.exists(wds_data_folder):
  os.makedirs(wds_data_folder)
# mc : monte-carlo

# number of layouts to generate as the training data for regression
# to build the power distribution surface

# mc_layout.dat file stores layouts only with 0s and 1s. 0 means no turbine here. 1 means one turbine here.
# The above file is used to generate wind power distribution.
# Each file has 10000 lines. Each line is layout.
# gen_mc_grid_with_NA_loc function generates these two files.
train_mc_layouts = LayoutGridMCGenerator.gen_mc_grid(rows_cells,
                                                     cols_cells,
                                                     n_mc_samples,
                                                     wt_N,
                                                     "{}/mc_layout.dat".format(wds_data_folder))

In [ ]:
# transformer dataset preperation



ind_to_xy_dic = {}
for ind in range(10 * 10):
  r_i = np.floor(ind / 10)
  c_i = np.floor(ind - r_i * 10)
  x_position = c_i * 400.0 + 200.0
  y_position = r_i * 400.0 + 200.0
  ind_to_xy_dic[ind] = np.array([x_position, y_position])
      
start_time = time.time()
encode_list = []
decode_list = []
op_list = []
for i in range(n_mc_samples):
  print("i", i)
  layout = train_mc_layouts[i]
  turbine_pos_array = np.zeros(50, dtype=np.int32)
  null_turbine_pos_array = np.zeros(50, dtype=np.int32)

  j = 0
  k = 0
  for ind in range(100):
    if(layout[ind]==1):
      turbine_pos_array[j] = ind
      j += 1
    else:
      null_turbine_pos_array[k]=ind
      k += 1

  true_aep = wfg.AEP(layout, 50)
  for j in range(100): # change this for number of samples for each layout
    turbine_pos = turbine_pos_array[np.random.randint(0, 50)]
    null_turbine_pos = null_turbine_pos_array[np.random.randint(0, 50)]
    changed_layout = np.copy(layout)
    changed_layout[turbine_pos] = 0
    changed_layout[null_turbine_pos] = 1
    aep_change = wfg.AEP(changed_layout, 50) - true_aep
    encode_list.append(np.array([ind_to_xy_dic[key] for key in turbine_pos_array]))
    decode_list.append(np.array([ind_to_xy_dic[turbine_pos], ind_to_xy_dic[null_turbine_pos]]))
    op_list.append(aep_change)

end_time = time.time()
print("time_took: ", end_time-start_time)

print(len(encode_list), len(op_list), len(decode_list))
# for i in range(len(encode_list)):
#   print(encode_list[i].shape, decode_list[i].shape, op_list[i])

i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
i 12
i 13
i 14
i 15
i 16
i 17
i 18
i 19
i 20
i 21
i 22
i 23
i 24
i 25
i 26
i 27
i 28
i 29
i 30
i 31
i 32
i 33
i 34
i 35
i 36
i 37
i 38
i 39
i 40
i 41
i 42
i 43
i 44
i 45
i 46
i 47
i 48
i 49
i 50
i 51
i 52
i 53
i 54
i 55
i 56
i 57
i 58
i 59
i 60
i 61
i 62
i 63
i 64
i 65
i 66
i 67
i 68
i 69
i 70
i 71
i 72
i 73
i 74
i 75
i 76
i 77
i 78
i 79
i 80
i 81
i 82
i 83
i 84
i 85
i 86
i 87
i 88
i 89
i 90
i 91
i 92
i 93
i 94
i 95
i 96
i 97
i 98
i 99
i 100
i 101
i 102
i 103
i 104
i 105
i 106
i 107
i 108
i 109
i 110
i 111
i 112
i 113
i 114
i 115
i 116
i 117
i 118
i 119
i 120
i 121
i 122
i 123
i 124
i 125
i 126
i 127
i 128
i 129
i 130
i 131
i 132
i 133
i 134
i 135
i 136
i 137
i 138
i 139
i 140
i 141
i 142
i 143
i 144
i 145
i 146
i 147
i 148
i 149
i 150
i 151
i 152
i 153
i 154
i 155
i 156
i 157
i 158
i 159
i 160
i 161
i 162
i 163
i 164
i 165
i 166
i 167
i 168
i 169
i 170
i 171
i 172
i 173
i 174
i 175
i 176
i 177
i 178
i 179
i 180
i 181
i 182
i 183
i 184


In [ ]:
with open("op_train_1.ls", "rb") as fp:   # Unpickling
  op_list = pickle.load(fp)

In [ ]:
count_positive=0
count_negative=0
sum_positive=0
sum_negative=0
for i in range(len(op_list)):
  if(op_list[i] > 0.0):
    count_positive+=1
    sum_positive+=op_list[i]
  elif(op_list[i] < 0.0):
    # print(op_list[i])
    count_negative+=1
    sum_negative+=op_list[i]

print(sum_negative, sum_positive)
print(sum_positive/count_positive, sum_negative/count_negative)

-68788.8869302378 70069.67462585556
0.6971968182310359 -0.6913942380894917


In [ ]:
import pickle
with open("encode_train_2.ls", "wb") as fp:   #Pickling
  pickle.dump(encode_list, fp)
with open("decode_train_2.ls", "wb") as fp:   #Pickling
  pickle.dump(decode_list, fp)
with open("op_train_2.ls", "wb") as fp:   #Pickling
  pickle.dump(op_list, fp)
# with open("decode_train_1.ls", "rb") as fp:   # Unpickling
#   b = pickle.load(fp)

In [ ]:
# file name to store the wind power distribution SVR model
svr_model_filename = 'svr_model.svr'

# load Monte-Carlo layouts from a text file. 10000 random layouts
layouts = np.genfromtxt("{}/mc_layout.dat".format(wds_data_folder), delimiter=" ", dtype=np.int32)
# generate the location index coordinate and average power output at each location index coordinate
# location index coordinate : in the cells, the cell with index 1 has location index (0,0) and the cell 2 has (1,0)
# store the location index coordinate in x.dat and average power in y.dat
# wfg.mc_gen_xy(rows=rows_cells, cols=cols_cells, layouts=layouts, n=n_mc_samples, N=wt_N, xfname="{}/x.dat".format(wds_data_folder),
#                  yfname="{}/y.dat".format(wds_data_folder))

# # read index location coordinates
# x_original = pd.read_csv("{}/x.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells, delim_whitespace=True, dtype=np.float32)
# x_original = x_original.values

# # read the power output of each index location coordinate
# y_original = pd.read_csv("{}/y.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells, delim_whitespace=True, dtype=np.float32)
# y_original = y_original.values.flatten()

# # create a SVR object and specify the kernal and other parameters
# svr_model = SVR(kernel='rbf', C=2000.0, gamma=0.3, epsilon=.1)
# # build the SVR power distribution model
# svr_model.fit(x_original, y_original)

# # save the SVR model to a file
# pickle.dump(svr_model, open("{}/{}".format(wds_data_folder,svr_model_filename), 'wb'))

# This is how to load SVR model from a file
svr_model = pickle.load(open("{}/{}".format(wds_data_folder,svr_model_filename), 'rb'))


**Run SUGGA method**

In [ ]:
for i in range(0, run_times):  # run times
  print("run times {} ...".format(i))
  wfg.load_init_pop("{}/init_{}.dat".format(init_pops_data_folder,i))
  run_time = wfg.sugga_genetic_algo(i,svr_model=svr_model,result_folder=sg_result_folder)
  result_arr[i, 0] = run_time
time_stamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = "{}/result_sugga_{}.dat".format(sg_result_folder,time_stamp)
np.savetxt(filename, result_arr, fmt='%f', delimiter="  ")

run times 0 ...
Support vector regression guided genetic algorithm starts....
generation 0...
best layout for generation  [1 0 1 0 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1 1 1 0 1 0 1
 0 1 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1
 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 1 1 1 1 1 1 0]
496.5705033874512
generation 1...
best layout for generation  [1 0 1 1 1 1 0 1 1 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0
 0 1 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1
 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1]
497.44304809570315
generation 2...
best layout for generation  [1 1 0 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0
 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0
 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 1]
498.02536834716796
generation 3...
best layout for generation  [1 1 0 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0
 0 

In [ ]:
def layout_to_xy(layout):
  xy_position = np.zeros((2, 50), dtype=np.float32)
  ind_pos = 0
  for ind in range(10 * 10):
    if layout[ind] == 1:
      r_i = np.floor(ind / 10)
      c_i = np.floor(ind - r_i * 10)
      xy_position[0, ind_pos] = c_i * 400.0 + 200.0
      xy_position[1, ind_pos] = r_i * 400.0 + 200.0
      # ind_position[ind_pos] = ind
      ind_pos += 1
  return xy_position

In [3]:
wind_data_file_name = "merged_wind_data.csv" # 2007, 2008, 2009, 2013, 2014, 2015, 2017
wfg.wind_data_file_name = wind_data_file_name
layouts_fname = "data/results/simple_scan/"+"sugga_best_layouts_N50_0.dat"
layouts = np.genfromtxt(layouts_fname, delimiter=" ", dtype=np.int32)
# print(layouts)
wfg.AEP(layouts[len(layouts)-1], 50)

NameError: ignored

In [ ]:
print(layouts[1])
xy = layout_to_xy(layouts[len(layouts)-1])
print(xy)

coords = {'x': xy[0],
          'y': xy[1]}
ddf = pd.DataFrame(coords)
print(ddf)
ddf.to_csv('final_xy.csv', index=False)

[1 0 1 1 1 1 0 1 1 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0
 0 1 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1
 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1]
[[ 200.  600. 1000. 1400. 1800. 2200. 2600. 3000. 3400. 3800.  200. 1400.
  2600. 3800.  200. 1000. 2200. 1800. 3400. 3800.  200. 1000. 2600. 3800.
   200. 1000. 1800. 2200. 3800.  200. 1400. 2600. 3800.  200. 1000. 1800.
  3000. 3800.  200. 3800.  200.  600. 1000. 1400. 1800. 2200. 2600. 3000.
  3400. 3800.]
 [ 200.  200.  200.  200.  200.  200.  200.  200.  200.  200.  600.  600.
   600.  600. 1000. 1000. 1000. 1400. 1400. 1400. 1800. 1800. 1800. 1800.
  2200. 2200. 2200. 2200. 2200. 2600. 2600. 2600. 2600. 3000. 3000. 3000.
  3000. 3000. 3400. 3400. 3800. 3800. 3800. 3800. 3800. 3800. 3800. 3800.
  3800. 3800.]]
         x       y
0    200.0   200.0
1    600.0   200.0
2   1000.0   200.0
3   1400.0   200.0
4   1800.0   200.0
5   2200.0   200.0
6   2600.0   200.0
7   3000.0   200.0
8 

In [1]:
def draw_points(points, name):
    plt.scatter(points[0], points[1], s=10)
    plt.title(name, fontsize=19)
    plt.xlabel("x-axis", fontsize=10)
    plt.ylabel("y-axis", fontsize=10)
    plt.tick_params(axis='both', which='major', labelsize=9)
    plt.show()

In [2]:
draw_points(xy, "simple_scan")

NameError: ignored

In [ ]:
# df = pd.read_csv(wind_data_file_name)
# print(df)
# wind_resource = df[['drct', 'sped']].to_numpy(dtype = np.float32)
# print(wind_resource.shape)

In [ ]:
# # Merging files 
# fnames = [2007, 2008, 2009, 2013, 2014, 2015, 2017]
# df = []
# for i in range(len(fnames)):
#   file_name = "wind_data_"+str(fnames[i])+".csv"
#   df.append(pd.read_csv(file_name))

In [ ]:
# merged_df = pd.concat(df)
# print(merged_df)

                   date   drct       sped
0      2007-01-01 00:20  290.0  12.829798
1      2007-01-01 00:50  290.0  15.842659
2      2007-01-01 01:20  290.0  15.270610
3      2007-01-01 01:50  300.0  13.332094
4      2007-01-01 02:20  290.0  12.639211
...                 ...    ...        ...
15902  2017-12-30 21:50  220.0   1.974951
15903  2017-12-30 22:20  180.0   2.330232
15904  2017-12-30 22:50  160.0   5.427657
15905  2017-12-30 23:20  170.0   6.016653
15906  2017-12-30 23:50  150.0   4.661694

[113113 rows x 3 columns]


In [ ]:
# merged_df.to_csv("merged_wind_data.csv", index=False)

In [ ]:
# print(pd.read_csv("merged_wind_data.csv"))

                    date   drct       sped
0       2007-01-01 00:20  290.0  12.829798
1       2007-01-01 00:50  290.0  15.842659
2       2007-01-01 01:20  290.0  15.270610
3       2007-01-01 01:50  300.0  13.332094
4       2007-01-01 02:20  290.0  12.639211
...                  ...    ...        ...
113108  2017-12-30 21:50  220.0   1.974951
113109  2017-12-30 22:20  180.0   2.330232
113110  2017-12-30 22:50  160.0   5.427657
113111  2017-12-30 23:20  170.0   6.016653
113112  2017-12-30 23:50  150.0   4.661694

[113113 rows x 3 columns]


In [ ]:
#Toy DL example not worked as expected


# n = 14
# x_max = 20
# y_max = 20
# min_dist = 5*5
# n_epochs = 10000
# min_dist_param = 1
# positive_param = 500


# def pairwise_distances(x, y=None):
#     '''
#     Input: x is a Nxd matrix
#            y is an optional Mxd matirx
#     Output: dist is a NxM matrix where dist[i,j] is the square norm between x[i,:] and y[j,:]
#             if y is not given then use 'y=x'.
#     i.e. dist[i,j] = ||x[i,:]-y[j,:]||^2
#     '''
#     x_norm = (x**2).sum(1).view(-1, 1)
#     if y is not None:
#         y_t = torch.transpose(y, 0, 1)
#         y_norm = (y**2).sum(1).view(1, -1)
#     else:
#         y_t = torch.transpose(x, 0, 1)
#         y_norm = x_norm.view(1, -1)
    
#     dist = x_norm + y_norm - 2.0 * torch.mm(x, y_t)
#     # Ensure diagonal is zero if x=y
#     # if y is None:
#     #     dist = dist - torch.diag(dist.diag)
#     return torch.clamp(dist, 0.0, np.inf)

# def check_min_dist(points, min_dist):
#   distances = pairwise_distances(points)
#   for i in range(distances.shape[0]):
#     distances[i][i] = min_dist+1
#   if(distances.min() >= min_dist):
#     return True
#   else:
#     return False

# def initialize(n, x_max, y_max, min_dist):
#   n_valid = 1
#   points = torch.tensor([[-x_max, -y_max]]) * torch.rand(1, 2) + torch.tensor([[x_max, y_max]])
#   while(n_valid<n):
#     new_point = torch.tensor([[-x_max, -y_max]]) * torch.rand(1, 2) + torch.tensor([[x_max, y_max]])
#     distance = ((points - new_point)**2).sum(axis=1)
#     if(distance.min() >= min_dist):
#       points = torch.cat((points, new_point), 0)
#       n_valid+=1
#   return points


def draw_points(points, name):
    plt.scatter(points[:,0], points[:,1], s=10)
    plt.title(name, fontsize=19)
    plt.xlabel("x-axis", fontsize=10)
    plt.ylabel("y-axis", fontsize=10)
    plt.tick_params(axis='both', which='major', labelsize=9)
    plt.show()

# def obj_fun(points):
#   return torch.sum(points)

# def loss_fun(points, min_dist_param, positive_param, min_dist, x_max, y_max):
#   relu_fun = nn.ReLU()
#   max = torch.tensor([[x_max, y_max]])
#   # print(points-max)
#   # print(relu_fun(min_dist - pairwise_distances(points)))
#   return -1 *  torch.sum(points) + min_dist_param * torch.sum(relu_fun(min_dist - pairwise_distances(points))) + positive_param * torch.sum(-1 * relu_fun(points)) + positive_param * torch.sum(relu_fun(points-max))

# points = initialize(n, x_max, y_max, min_dist)
# draw_points(points, "Before Optimization")
# points.requires_grad=True
# optimizer = optim.Adam([points], lr=0.001)
# check_min_dist(points, min_dist)
# print(points)


# for epoch in range(n_epochs):
#   loss = loss_fun(points, min_dist_param, positive_param, min_dist, x_max, y_max)
#   # print(epoch, obj_fun(points))
#   optimizer.zero_grad()
#   loss.backward()
#   # print(points.grad)
#   optimizer.step()
#   # print(points)

# print(obj_fun(points))
# print(check_min_dist(points, min_dist))
# draw_points(points.detach().numpy(), "after_optimization")
# print(points)


# points = torch.tensor([[-5, -5]]) * torch.rand(5, 2) + torch.tensor([[5, 5]]) 
# print(points)
# distance = points - torch.tensor([[2, 3]])
# distance = distance * distance
# print(distance)
# print(distance.sum(axis=1))

In [ ]:
positionX = np.random.randint(0, 10, size=(2, 3))
print(positionX)
positionX[0, [0,2]]=[1, 3]
print(positionX)
# print(np.sum(positionX, axis=0))


In [ ]:
import numpy as np
p = np.asarray([[1,2],[3,4],[5,6]])
q = p[0]
q[0] = 57
print(p)



[[57  2]
 [ 3  4]
 [ 5  6]]
